## Importing libraries

In [1]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

## Configuring workspace details

In [ ]:
subscription_id="-----------------"
resource_group="-----------------"
workspace="-----------------"

In [5]:
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

## Registering the Model 

### First we should run the "convert_to_MLflow.py" script so that we can convert our model to a MLflow model

### 1- Now Registering the Model

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path="/home/azureuser/cloudfiles/code/Users/<User-name>/Real-time-spam-detection-with-Azure/spam_detection_mlflow",
    type=AssetTypes.MLFLOW_MODEL,
    name="spam-detection-model",
    description="Custom model to predict spams and non-spams"
    )

ml_client.models.create_or_update(file_model)

Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'spam-detection-model', 'description': 'Custom model to predict spams and non-spams', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/97f7cf11-b890-412c-be78-d508db5a61e5/resourceGroups/ml-project/providers/Microsoft.MachineLearningServices/workspaces/cloud-model-deployement/models/spam-detection-model/versions/2', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/spam-detection/code/Users/amine.77_5-allali/Real-time-spam-detection-with-Azure', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7efd72703a60>, 'serialize': <msrest.serialization.Serializer object at 0x7efd72703df0>, 'version': '2', 'latest_version': None, 'path': 'azureml://subscriptions/97f7cf11-b890-412c-be78-d508db5a61e5/resourceGroups/ml-project/workspaces/cloud-model-deployem

### 2- Registering the environment

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="/home/azureuser/cloudfiles/code/Users/<User-name>/Real-time-spam-detection-with-Azure/spam_detection_mlflow/conda.yaml",
    name="my-env",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env_docker_conda)

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'my-env', 'description': 'Environment created from a Docker image plus Conda environment.', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/97f7cf11-b890-412c-be78-d508db5a61e5/resourceGroups/ml-project/providers/Microsoft.MachineLearningServices/workspaces/cloud-model-deployement/environments/my-env/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/spam-detection/code/Users/amine.77_5-allali/Real-time-spam-detection-with-Azure', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7efd733884c0>, 'serialize': <msrest.serialization.Serializer object at 0x7efd72604cd0>, 'version': '1', 'conda_file': {'c

## Defining endpoint 

In [12]:
endpoint_name = "spam-detector-endpoint-v1"

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="this is an online endpoint that tells if a text is spam or not",
    auth_mode="key",
)

In [13]:
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://spam-detector-endpoint-v1.francecentral.inference.ml.azure.com/score', 'openapi_uri': 'https://spam-detector-endpoint-v1.francecentral.inference.ml.azure.com/swagger.json', 'name': 'spam-detector-endpoint-v1', 'description': 'this is an online endpoint that tells if a text is spam or not', 'tags': {}, 'properties': {'createdBy': 'amine allali', 'createdAt': '2024-12-07T18:17:24.288337+0000', 'lastModifiedAt': '2024-12-07T18:17:24.288337+0000', 'azureml.onlineendpointid': '/subscriptions/97f7cf11-b890-412c-be78-d508db5a61e5/resourcegroups/ml-project/providers/microsoft.machinelearningservices/workspaces/cloud-model-deployement/onlineendpoints/spam-detector-endpoint-v1', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/97f7cf11-b890-412c-be78-d508db5a61e5/providers/Microsoft.MachineLearningServices/locations/francecentral/mfeOperationsStatus/oeidp:73d

In [ ]:
model = "azureml:spam-detection-model:1"
env = "azureml:my-env:1"

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="/home/azureuser/cloudfiles/code/Users/<User-name>/Real-time-spam-detection-with-Azure/onlinescoring", 
        scoring_script="score.py"
    ),
    instance_type="Standard_DS3_v2",
    instance_count=1,
)